In [0]:
rm -rf RAVDESS 

In [0]:
# helper to download the Data Set from Google Drive

import requests
import os

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)


In [0]:
# Download the pre processed data set
# file_id = '1a2TRcqj5ySMWwetfU3Zgq4FE-highOKK'
file_id = '1l8pPuX0SKPx9wnIfYeP7NZ-jTsNBd4nu'


destination = 'RAVDESS.zip'

if not os.path.exists(destination):
    download_file_from_google_drive(file_id, destination)

In [4]:
!ls -lh

total 154M
-rw-r--r-- 1 root root 154M Dec 12 08:38 RAVDESS.zip
drwxr-xr-x 1 root root 4.0K Dec  6 16:53 sample_data


In [5]:
!unzip -n RAVDESS.zip

Archive:  RAVDESS.zip
   creating: RAVDESS/train_valid/
   creating: RAVDESS/train_valid/test/
   creating: RAVDESS/train_valid/test/surprised/
  inflating: RAVDESS/train_valid/test/surprised/03-01-08-01-02-01-09-300.png  
  inflating: RAVDESS/train_valid/test/surprised/03-01-08-01-01-01-13-0.png  
  inflating: RAVDESS/train_valid/test/surprised/03-01-08-02-02-01-04-100.png  
  inflating: RAVDESS/train_valid/test/surprised/03-01-08-01-01-02-14-0.png  
  inflating: RAVDESS/train_valid/test/surprised/03-01-08-01-02-02-05-0.png  
  inflating: RAVDESS/train_valid/test/surprised/03-01-08-01-02-02-01-100.png  
  inflating: RAVDESS/train_valid/test/surprised/03-01-08-01-01-01-06-0.png  
  inflating: RAVDESS/train_valid/test/surprised/03-01-08-02-02-02-11-97.png  
  inflating: RAVDESS/train_valid/test/surprised/03-01-08-01-02-01-08-223.png  
  inflating: RAVDESS/train_valid/test/surprised/03-01-08-02-01-02-02-100.png  
  inflating: RAVDESS/train_valid/test/surprised/03-01-08-01-01-01-08-223.pn

In [6]:
!ls RAVDESS/train_valid/test -lh

total 32K
drwxr-xr-x 2 root root 4.0K Dec  8 14:30 angry
drwxr-xr-x 2 root root 4.0K Dec  8 14:30 calm
drwxr-xr-x 2 root root 4.0K Dec  8 14:30 disgust
drwxr-xr-x 2 root root 4.0K Dec  8 14:30 fearful
drwxr-xr-x 2 root root 4.0K Dec  8 14:30 happy
drwxr-xr-x 2 root root 4.0K Dec  8 14:30 neutral
drwxr-xr-x 2 root root 4.0K Dec  8 14:30 sad
drwxr-xr-x 2 root root 4.0K Dec  8 14:30 surprised


In [7]:
# Base imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import re
import os
import skimage.io as io
from IPython.display import Image, display
from sklearn.model_selection import train_test_split
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# keras imports
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv2D, Flatten, \
    Conv1D, Input,BatchNormalization, MaxPooling2D
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras import regularizers


# Set seed value
seed_value = 0
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
import tensorflow as tf
tf.set_random_seed(seed_value)
from keras import backend as K
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

Using TensorFlow backend.


In [0]:
def log(model, history,):
    files = os.listdir()
    count = len(list(filter(lambda x: x[:len('log')] =='log', files)))+1
    print(count)
    with open('log_'+str(count)+ '.txt','w') as fh:
        # Pass the file handle in as a lambda function to make it callable
        model.summary(print_fn=lambda x: fh.write(x + '\n'))
        acc = history.history['acc']
        val_acc = history.history['val_acc']

        loss = history.history['loss']
        val_loss = history.history['val_loss']
        for i in range(len(acc)):
            print(i,acc[i],val_acc[i], loss[i],val_loss[i], sep=',', file=fh)

In [0]:
# create directory path helper variables
base_dir = 'RAVDESS/train_valid/'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'valid')
test_dir = os.path.join(base_dir, 'test')

In [10]:
print("Number of original train images: " + str(sum([len(files) for r, d, files in os.walk(train_dir)])))
print("Number of original validation images: " + str(sum([len(files) for r, d, files in os.walk(validation_dir)])))
print("Number of original test images: " + str(sum([len(files) for r, d, files in os.walk(test_dir)])))

Number of original train images: 4931
Number of original validation images: 615
Number of original test images: 621


In [0]:
img_height=128
img_width=200

In [0]:
emotions = {
    "neutral": 0, 
    "calm": 1, 
    "happy": 2, 
    "sad": 3, 
    "angry": 4, 
    "fearful":5, 
    "disgust":6 , 
    "surprised":7}
# Train
X_train = []
y_train = []
for r, d, files in os.walk(train_dir):
    if files != [] :
        emotion = r[r.rfind('/')+1:]
        emotion = emotions[emotion]
        count = 0
        for file in files: 
            img = io.imread(r+ '/' +file , as_gray=True)
            img = img.reshape([img_height, img_width, 1])
            X_train.append(img)
            y_train.append(emotion)
            count += 1
            # if count > 200: break
X_train = np.array(X_train)
y_train = np.array(y_train)
y_train = np_utils.to_categorical(y_train, 8)
# Valid
X_valid = []
y_valid = []
for r, d, files in os.walk(validation_dir):
    if files != [] :
        emotion = r[r.rfind('/')+1:]
        emotion = emotions[emotion]
        for file in files: 
            img = io.imread(r+ '/' +file , as_gray=True)
            img = img.reshape([img_height, img_width, 1])
            X_valid.append(img)
            y_valid.append(emotion)
X_valid = np.array(X_valid)
y_valid = np.array(y_valid)
y_valid = np_utils.to_categorical(y_valid, 8)
# Test
X_test = []
y_test = []
for r, d, files in os.walk(test_dir):
    if files != [] :
        emotion = r[r.rfind('/')+1:]
        emotion = emotions[emotion]
        for file in files: 
            img = io.imread(r+ '/' +file , as_gray=True)
            img = img.reshape([img_height, img_width, 1])
            X_test.append(img)
            y_test.append(emotion)
X_test = np.array(X_test)
y_test = np.array(y_test)
y_test = np_utils.to_categorical(y_test, 8)

In [0]:
ind = np.arange(len(X_train))
np.random.shuffle(ind)
X_train = X_train[list(ind),:]
y_train = y_train[list(ind),:]

ind = np.arange(len(X_valid))
np.random.shuffle(ind)
X_valid = X_valid[list(ind),:]
y_valid = y_valid[list(ind),:]


In [0]:
batch_size = 128
num_classes = 8
epochs = 100
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(10,10), input_shape=(128,200,1)))
model.add(MaxPooling2D(pool_size=(2,2), padding='valid'))
model.add(Conv2D(filters=32, kernel_size=(10,10), input_shape=(128,200,1)))
model.add(MaxPooling2D(pool_size=(2,2), padding='valid'))
model.add(Conv2D(filters=32, kernel_size=(10,10), input_shape=(128,200,1)))
'''model.add(Conv2D(filters=32, kernel_size=(10,10),activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(MaxPooling2D(pool_size=(2,2), padding='valid'))
model.add(Conv2D(filters=32, kernel_size=(10,10),activation='relu',kernel_regularizer=regularizers.l2(0.01),))
model.add(Conv2D(filters=32, kernel_size=(10,10),activation='relu',kernel_regularizer=regularizers.l2(0.01),))
model.add(MaxPooling2D(pool_size=(2,2), padding='valid', input_shape=(128,200,1)))
'''
model.add(Flatten())
model.add(Dense(128, activation='relu',kernel_regularizer=regularizers.l2(0.01), input_shape=(128,200,1)))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(32))
model.add(Dropout(0.4))
model.add(Dense(num_classes,activation='softmax'))
model.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()
callbacks = [EarlyStopping(monitor='val_acc', patience=5, verbose=0)]

In [0]:
batch_size = 128
num_classes = 8
epochs = 100
model = Sequential() 
model.add(Conv2D(32, kernel_size=(5, 5), activation='relu', input_shape=(128,200,1,))) 
model.add(Dropout(0.5))
model.add(Conv2D(64, (5, 5), activation='relu')) 
model.add(Dropout(0.5))
model.add(Conv2D(128, (5, 5), activation='relu')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax')) 
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'] ) #
callbacks = [EarlyStopping(monitor='val_loss', patience=8, verbose=0)]
sum = 0
for i in model.get_weights():
    sum += (np.size(i))
print('Number of parameters: ',sum)
history = model.fit(x=X_train, y=y_train, batch_size=batch_size, epochs=epochs,validation_data=(X_valid,y_valid),callbacks=callbacks,verbose=1)
log(model, history,)
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [0]:
batch_size = 128
num_classes = 8
epochs = 100
model = Sequential() 
model.add(Conv2D(64, kernel_size=(2, 2),  activation='relu' , input_shape=(128,200,1,),kernel_regularizer=regularizers.l1_l2(0.00001,0.05))) # 
model.add(Dropout(0.35))
model.add(Conv2D(64, (2, 2), activation='relu',kernel_regularizer=regularizers.l1_l2(0.0001,0.005))) 
model.add(Dropout(0.35))
model.add(Conv2D(64, (2, 2), activation='relu',kernel_regularizer=regularizers.l1_l2(0.00001,0.005))) 
model.add(Dropout(0.35))
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Conv2D(64, (2, 2), activation='relu',kernel_regularizer=regularizers.l1_l2(0.0001,0.005))) 
model.add(Dropout(0.4))
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Flatten())
model.add(Dense(128, activation='relu',kernel_regularizer=regularizers.l1_l2(0,0.000001))) # 
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax')) 
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'] ) #
callbacks = [EarlyStopping(monitor='val_loss', patience=8, verbose=0)]
sum = 0
for i in model.get_weights():
    sum += (np.size(i))
print('Number of parameters: ',sum)
history = model.fit(x=X_train, y=y_train, batch_size=batch_size, epochs=epochs,validation_data=(X_valid,y_valid),callbacks=callbacks,verbose=1)
log(model, history,)
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

Number of parameters:  11847304
Train on 4931 samples, validate on 615 samples
Epoch 1/100
4931/4931 [==============================] - 15s 3ms/step - loss: 3.4954 - acc: 0.1397 - val_loss: 2.9547 - val_acc: 0.1398
Epoch 2/100
4931/4931 [==============================] - 13s 3ms/step - loss: 2.8344 - acc: 0.1470 - val_loss: 2.7411 - val_acc: 0.1415
Epoch 3/100
4931/4931 [==============================] - 13s 3ms/step - loss: 2.6687 - acc: 0.1399 - val_loss: 2.6218 - val_acc: 0.1545
Epoch 4/100
4931/4931 [==============================] - 13s 3ms/step - loss: 2.5677 - acc: 0.1489 - val_loss: 2.5369 - val_acc: 0.1545
Epoch 5/100
4931/4931 [==============================] - 13s 3ms/step - loss: 2.4960 - acc: 0.1584 - val_loss: 2.4828 - val_acc: 0.1821
Epoch 6/100
4931/4931 [==============================] - 13s 3ms/step - loss: 2.4300 - acc: 0.1785 - val_loss: 2.4267 - val_acc: 0.2423
Epoch 7/100
4931/4931 [==============================] - 13s 3ms/step - loss: 2.3745 - acc: 0.1979 - val_